# AIDA Freediving Records

The Scope of that project is explore and display all freediving records from one of the world organization of freediving, AIDA in a series of D3.js visualizations. Such plateform could be extended to other datasets that share similar types of information (athlete name, country, record values (time, distance), location of the event, date).

1- Scraping the data from the website

2- Data preparation / cleaning / extension (separate name / country, get GPS locations, get gender...)

3- Early data exploration

4- D3.js data visualizations

# Scraping the data from www.aidainternational.org

Modules lxml and beautifulsoup parse the html in the 'page' variable returned from the website and store it in Beautiful Soup format. Import the library used to query a website and return the html to the variable 'page'. Convert list to dataframe with Pandas.

In [14]:
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import requests as rq

In [15]:
url = 'https://www.aidainternational.org/Ranking/Rankings?page=1'
page = rq.get(url)
soup = BeautifulSoup(page.content, "lxml") 

#Generate lists
A=[]
B=[]
C=[]
D=[]
E=[]
F=[]
G=[]
H=[]
for row in soup.table.tbody.findChildren('tr'):
    cells = row.findAll('td')
    A.append(cells[0].find(text=True))
    B.append(cells[1].find(text=True))
    C.append(cells[2].find(text=True))
    D.append(cells[3].find(text=True))
    E.append(cells[4].find(text=True))
    F.append(cells[5].find(text=True))
    G.append(cells[6].find(text=True))
    H.append(cells[7].find(text=True))

#Convert list to data frame
df=pd.DataFrame(A,columns=['#'])
df['Athlete_Name']=B
df['Result']=C
df['Announced']=D
df['Points']=E
df['Penalties']=F
df['Date']=G
df['Place']=H


df

,#,Athlete_Name,Result,Announced,Points,Penalties,Date,Place
0,1.,Alexey MOLCHANOV,128,128,127,1,11/20/2012,Suunto Vertical Blue 2012
1,2.,Alexey MOLCHANOV,128,128,127,1,11/20/2012,Suunto Vertical Blue 2012
2,3.,Alexey MOLCHANOV,128,128,127,1,11/20/2012,Suunto Vertical Blue 2012
3,4.,Alexey MOLCHANOV,128,128,128,0,9/15/2013,AIDA INDIVIDUAL DEPHT WORLD CHAMPIONSHIP 2013
4,5.,Alexey MOLCHANOV,128,128,0,0,9/9/2013,2013 Kalamata Open
5,6.,Guillaume NÉRY,126,126,126,0,9/5/2015,2015 AIDA Depth World Championships Pre compet...
6,7.,Alexey MOLCHANOV,126,126,126,0,11/20/2012,Suunto Vertical Blue 2012
7,8.,Alexey MOLCHANOV,126,126,126,0,11/20/2012,Suunto Vertical Blue 2012
8,9.,Alexey MOLCHANOV,126,126,126,0,11/20/2012,Suunto Vertical Blue 2012
9,10.,Guillaume NÉRY,125,125,125,0,9/9/2013,2013 Kalamata Open


In [16]:
df.shape 

(20, 8)

In [3]:
#page = rq.get('https://www.aidainternational.org/Ranking/Rankings?page=1')

In [4]:
#page.status_code 

200

NB: a status code starting with a 2 generally indicates success, and a code starting with a 4 or a 5 indicates an error.

In [370]:
#page.content

In [7]:
#soup = BeautifulSoup(page.content, "lxml")
#soup

In [373]:
#print(soup.prettify())

The text data 'td' I am interested in are in the body 'tbody' of a table, two tags down with each row 'tr'

In [374]:
#soup.table.tbody.findChildren('tr')

Dataframe 'df' contains 20 first rows from page 1. Now, I will like to collect data from all 336 pages available for the discipline CWT - constant weight

-

# Webscraping multiple pages

In [14]:
#base_url = 'https://www.aidainternational.org/Ranking/Rankings?page='
#r = rq.get(base_url)

In [77]:
#for i in range(2):      # Number of pages plus one 
#    url = '{}{}'.format(base_url, i)
#    r = rq.get(url)
#    new_soup = BeautifulSoup(r.content, "lxml")    

In [277]:
#url_list = ['{}{}'.format(base_url, i) for i in range(1,5)]
#url_list

['https://www.aidainternational.org/Ranking/Rankings?page=1',
 'https://www.aidainternational.org/Ranking/Rankings?page=2',
 'https://www.aidainternational.org/Ranking/Rankings?page=3',
 'https://www.aidainternational.org/Ranking/Rankings?page=4']

Generate a list with all Urls e.g. max_pages=8

In [171]:
base_url = 'https://www.aidainternational.org/Ranking/Rankings?page='

def url_generator(max_pages):
    urls = ['{}{}'.format(base_url, i) for i in range(1, max_pages + 1)]
    return urls

In [173]:
urls 

['https://www.aidainternational.org/Ranking/Rankings?page=1',
 'https://www.aidainternational.org/Ranking/Rankings?page=2',
 'https://www.aidainternational.org/Ranking/Rankings?page=3',
 'https://www.aidainternational.org/Ranking/Rankings?page=4',
 'https://www.aidainternational.org/Ranking/Rankings?page=5',
 'https://www.aidainternational.org/Ranking/Rankings?page=6']

In [175]:
#def make_soup(max_pages):
    page = rq.get(urls)
    new_soup = BeautifulSoup(page.content)
    test = new_soup.table.tbody.findAll('td')
return test

IndentationError: unexpected indent (<ipython-input-175-250b23dfd9af>, line 2)

In [167]:
make_soup(3)

Processing 1...
Processing 2...
Processing 3...


[<td>41.</td>,
 <td><a href="/AthleteDetails?athleteId=00000000-0000-0000-0000-000000000802"><strong>William TRUBRIDGE</strong> (New Zealand)</a></td>,
 <td>\r\n116<span> m</span>\n</td>,
 <td>116</td>,
 <td>116</td>,
 <td>0</td>,
 <td>5/25/2014</td>,
 <td>Caribbean Cup 2014</td>,
 <td>42.</td>,
 <td><a href="/AthleteDetails?athleteId=00000000-0000-0000-0000-000000000802"><strong>William TRUBRIDGE</strong> (New Zealand)</a></td>,
 <td>\r\n116<span> m</span>\n</td>,
 <td>116</td>,
 <td>116</td>,
 <td>0</td>,
 <td>4/27/2015</td>,
 <td>Suunto Vertical Blue 2015</td>,
 <td>43.</td>,
 <td><a href="/AthleteDetails?athleteId=00000000-0000-0000-0000-0000000000ae"><strong>Ryuzo SHINOMIYA</strong> (Japan)</a></td>,
 <td>\r\n115<span> m</span>\n</td>,
 <td>115</td>,
 <td>115</td>,
 <td>0</td>,
 <td>4/27/2010</td>,
 <td>Vertical Blue 2010 - the Suunto Dive-Off</td>,
 <td>44.</td>,
 <td><a href="/AthleteDetails?athleteId=00000000-0000-0000-0000-0000000001dd"><strong>Guillaume N\xc9RY</strong> (Fran

In [179]:
from bs4 import BeautifulSoup
import urllib
import lxml.html
import time
import os

base_url = 'https://www.aidainternational.org/Ranking/Rankings?page='

def scraper(max_pages):
    
    '''This function crawls the AIDA website, gathering urls to the freediving 
    records pages for CWT discipline. The url list fed into it is a list of all 
    the pages from the website that result from the search 'CWT' 
    (338 total, 20 records on each page).'''
    
    for i in range(1, max_pages + 1): # Number of pages plus one
        print "Processing {}...".format(i) 
        url = '{}{}'.format(base_url, i)
        page = rq.get(url)
        new_soup = BeautifulSoup(page.content, "lxml")
        
        data = {
            'a_#':[],
            'b_Name':[],
            'c_Result':[],
            'd_Announced':[],
            'e_Points':[],
            'f_Penalties':[],
            'g_Date':[],
            'h_Place':[]
            }
        
        rows = new_soup.table.tbody.findChildren('tr')
        for row in rows:
            cols = row.findAll('td')

            data['a_#'].append( cols[0].get_text().replace('\n', ' ').replace('\r', ' ') )
            data['b_Name'].append( cols[1].get_text().replace('\n', ' ').replace('\r', ' ') )
            data['c_Result'].append( cols[2].get_text().replace('\n', ' ').replace('\r', ' ') )
            data['d_Announced'].append( cols[3].get_text().replace('\n', ' ').replace('\r', ' ') )
            data['e_Points'].append( cols[4].get_text().replace('\n', ' ').replace('\r', ' ') )
            data['f_Penalties'].append( cols[5].get_text().replace('\n', ' ').replace('\r', ' ') )
            data['g_Date'].append( cols[6].get_text().replace('\n', ' ').replace('\r', ' ') )
            data['h_Place'].append( cols[7].get_text().replace('\n', ' ').replace('\r', ' ') )
            
        df = pd.DataFrame(data)

filename = '/Users/fabienplisson/Desktop/DeepPlay/deepplay/notebooks/results.txt'
f = open(filename, 'a')
f.write(str(df))
f.close()   

In [180]:
scraper(3)

Processing 1...
Processing 2...
Processing 3...


In [183]:
df

,#,Athlete_Name,Result,Announced,Points,Penalties,Date,Place
0,1.,Alexey MOLCHANOV,128,128,127,1,11/20/2012,Suunto Vertical Blue 2012
1,2.,Alexey MOLCHANOV,128,128,127,1,11/20/2012,Suunto Vertical Blue 2012
2,3.,Alexey MOLCHANOV,128,128,127,1,11/20/2012,Suunto Vertical Blue 2012
3,4.,Alexey MOLCHANOV,128,128,128,0,9/15/2013,AIDA INDIVIDUAL DEPHT WORLD CHAMPIONSHIP 2013
4,5.,Alexey MOLCHANOV,128,128,0,0,9/9/2013,2013 Kalamata Open
5,6.,Guillaume NÉRY,126,126,126,0,9/5/2015,2015 AIDA Depth World Championships Pre compet...
6,7.,Alexey MOLCHANOV,126,126,126,0,11/20/2012,Suunto Vertical Blue 2012
7,8.,Alexey MOLCHANOV,126,126,126,0,11/20/2012,Suunto Vertical Blue 2012
8,9.,Alexey MOLCHANOV,126,126,126,0,11/20/2012,Suunto Vertical Blue 2012
9,10.,Guillaume NÉRY,125,125,125,0,9/9/2013,2013 Kalamata Open
